
<a href="https://github.com/Filipecdj" rel="some text">
  <img src="https://uploads-ssl.webflow.com/61954f44e9b26e303bba74f5/61a7850e3a3a86e7e5ba1f35_logo_datarisk_header.png" height="100px" style="display: block; margin: auto;">
</a>


<h2 style="text-align:center">Case Engenheiro de dados</h2>
<p style="text-align:center;font-size:13px;"></p>
   
<h2 style="text-align:center">Filipe Carvalho de Jesus</h2>
<p style="text-align:center;font-size:13px;">Autor</p>

# Sumário

- [1. Questão 1](#1)<br>
- [2. Questão 2](#2)<br>
- [3. Questão 3](#3)<br>
- [4. Questão 4](#4)<br>
- [5. Questão 5](#5)<br>


# 1 - Configurando ambiente PySpark na máquina <a id="1"></a>

In [ ]:
# Instalações
!pip install pyspark -q

In [ ]:
# Bibliotecas
from pyspark.sql import SparkSession

# Extraindo informações diárias dos últimos 30 dias(úteis), através de um script Python.<a id="1"></a>

# Questão 1 - Extrair informações diárias de uma das ações abaixo dos últimos 30 dias(úteis):
1. ITUB - Itaú
2. BBD - Bradesco
3. MSFT - Microsoft
4. GOOG - Google
5. TSLA - Tesla

Bibliotecas utilizadas. Como foi solicitado no desafio, foi armazenado o token de API em um arquivo chave_api.py

In [1]:
import requests
import pandas as pd
from chave_api import chave
from pyspark.sql.functions import *
from pyspark.sql import SparkSession



Instanciando o objeto para capturar a chave de api 

In [2]:
chave_api = chave()
key = chave_api.get_key()

Listas criadas para parametrizar a captura de dados das respectivas ações: ITUB, BBD, MSFT, GOOG e TSLA

In [3]:
acoes = ['ITUB','BBD','MSFT','GOOG','TSLA']
lista_acoes = []

O Script abaixo, realiza a captura e a formatação dos dados em JSON das ações solicitadas pelo desafio. Onde é acessado a chave "Time Series (Daily)" do JSON, renomeia o index para "Data", seleciona as respectivas colunas, cria no dataframe uma colunas para indicar a ação, transforma o index em uma coluna, filtra os últimos 30 dias úteis e ao final todas elas são concatenadas em um dataframe Pandas.

In [4]:
for acao in acoes:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={acao}&outputsize=compact&apikey={key}'
    r = requests.get(url)
    data = r.json()

    
    time_series = data["Time Series (Daily)"] 

    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index.name = 'date'
    df.columns = ['open', 'high', 'low', 'close', 'adjusted close', 'volume', 'dividend amount', 'split coefficient']
    df = df.astype(float)
    df['Symbol']= acao
    df = df.reset_index()    
    df = df.head(30)        
    lista_acoes.append(df)

df_acoes = pd.concat(lista_acoes, ignore_index=True)


# Código em PySpark para realizar os questionários 2, 3, 4 e 5 

A configuração da sessão abaixo, está utilizando 3 threads juntamente com o arquivo .jar do PostreSQL. Isso, para realizar a conexão com o banco de dados Postgre local

In [5]:
spark = SparkSession.builder \
.master("local[3]")\
.appName("myApp") \
.config("spark.jars", "\postgresql-42.5.0.jar")\
.getOrCreate()

spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

Abaixo, é realizado a converção de um dataframe Pandas para um dataframe Pyspark, utilizando a função **createDataFrame** do PySpark. Após isso, é mostrado um exemplo do dataframe contendo as informações.

In [6]:
pysp_df = spark.createDataFrame(df_acoes)
pysp_df.show()

+----------+-----+------+------+-----+----------------+-----------+---------------+-----------------+------+
|      date| open|  high|   low|close|  adjusted close|     volume|dividend amount|split coefficient|Symbol|
+----------+-----+------+------+-----+----------------+-----------+---------------+-----------------+------+
|2023-04-28| 5.09| 5.165|  5.06| 5.15|            5.15|1.7423588E7|            0.0|              1.0|  ITUB|
|2023-04-27| 5.08|5.1475|  5.05| 5.14|            5.14|1.7235927E7|            0.0|              1.0|  ITUB|
|2023-04-26| 5.02|5.0582|  4.97| 4.99|            4.99|1.2525425E7|            0.0|              1.0|  ITUB|
|2023-04-25|  5.0|  5.05|  4.96| 5.02|            5.02|2.1134543E7|            0.0|              1.0|  ITUB|
|2023-04-24| 4.99| 5.055|  4.92|  5.0|             5.0|1.6361481E7|            0.0|              1.0|  ITUB|
|2023-04-21| 5.02|  5.03|  4.96| 5.03|            5.03|  5596926.0|            0.0|              1.0|  ITUB|
|2023-04-20| 4.97| 

# Questão 2 - Valores da média, o desvio padrão, o valor mínimo, os quartis da distribuição e o valor máximo dos últimos 30 dias(úteis).<a id="2"></a>

Filtro para responder a questão 2, onde é mostrado: Média, desvio padrão, valor mínimo, quartis e valor máximo

In [7]:
quest2 = pysp_df.groupBy("Symbol").agg(
    round(mean("close"),4).alias("media_acao"),
    round(stddev("close"),4).alias("desvio_padrao"),
    min("close").alias("valor_min"),
    percentile_approx("close", 0.25).alias("q1"),
    percentile_approx("close", 0.5).alias("q2"),
    percentile_approx("close", 0.75).alias("q3"),
    max("close").alias("valor_max"),
)

quest2.show()

+------+----------+-------------+---------+------+------+------+---------+
|Symbol|media_acao|desvio_padrao|valor_min|    q1|    q2|    q3|valor_max|
+------+----------+-------------+---------+------+------+------+---------+
|  ITUB|    4.8427|       0.2878|     4.33|  4.56|  4.83|  5.07|     5.26|
|   BBD|    2.6347|       0.1272|     2.38|  2.52|  2.63|  2.76|     2.83|
|  GOOG|   105.301|       2.1796|   101.32|104.22|105.12|106.42|   109.46|
|  MSFT|  284.8203|       8.3014|   272.23|279.43|284.34|288.45|   307.26|
|  TSLA|  182.4837|       13.217|   153.75|180.13|185.06|191.81|   207.46|
+------+----------+-------------+---------+------+------+------+---------+



# Questão 3 - Fazer uma ordenação dos dados e seleção dos 'n' maiores e menores volumes dos últimos 30 dias(úteis)<a id="3"></a>

O código abaixo, ordena o dataframe em ordem crescente com base no volume, selecionando os 2 maiores (df_maior) e menores (df_menor) volumes. É possivel alterar o valor de "n" para visualizar mais dados.

In [8]:

quest3 = pysp_df.orderBy("volume")

n = 2
df_maior = quest3.orderBy(desc("volume")).limit(n)
df_menor = quest3.limit(n)

Menores volumes comparando todas ações

In [9]:
df_menor.show(n)

+----------+----+----+-----+-----+--------------+---------+---------------+-----------------+------+
|      date|open|high|  low|close|adjusted close|   volume|dividend amount|split coefficient|Symbol|
+----------+----+----+-----+-----+--------------+---------+---------------+-----------------+------+
|2023-04-21|5.02|5.03| 4.96| 5.03|          5.03|5596926.0|            0.0|              1.0|  ITUB|
|2023-04-21|2.68|2.69|2.655| 2.69|          2.69|8210718.0|            0.0|              1.0|   BBD|
+----------+----+----+-----+-----+--------------+---------+---------------+-----------------+------+



Maiores volumes comparando todas ações

In [10]:
df_maior.show(n)

+----------+-------+------+------+------+--------------+------------+---------------+-----------------+------+
|      date|   open|  high|   low| close|adjusted close|      volume|dividend amount|split coefficient|Symbol|
+----------+-------+------+------+------+--------------+------------+---------------+-----------------+------+
|2023-04-20|166.165| 169.7|160.56|162.99|        162.99|2.10970819E8|            0.0|              1.0|  TSLA|
|2023-03-31| 197.53|207.79| 197.2|207.46|        207.46|1.70222118E8|            0.0|              1.0|  TSLA|
+----------+-------+------+------+------+--------------+------------+---------------+-----------------+------+



# Questão 4 - Faça a soma dos volumes de ITUB e BBD dos últimos 30 dias(úteis)<a id="4"></a>

O seguinte código, filtra o dataframe para mostrar somente as ações ITUB e BBD e realiza a soma total dos volumes. Ao final, é printado o resultado.

In [11]:
quest4 = pysp_df.filter(pysp_df.Symbol.isin('BBD','ITUB'))
soma_volumes = (quest4.select(sum('volume')).collect()[0][0])

print(f'Soma do volume BBD e ITUB: {soma_volumes}')

Soma do volume BBD e ITUB: 1591330590.0


Filtra o dataframe da ação ITUB para somar o volume dos últimos 30 dias úteis.

In [12]:
soma_itub = pysp_df.filter(pysp_df.Symbol.isin('ITUB'))
soma_volumes_itub = (soma_itub.select(sum('volume')).collect()[0][0])

print(f'Soma do volume ITUB: {soma_volumes_itub}')

Soma do volume ITUB: 811756436.0


Filtra o dataframe da ação BBD para somar o volume dos últimos 30 dias úteis.

In [13]:
soma_bbd = pysp_df.filter(pysp_df.Symbol.isin('BBD'))
soma_volumes_bbd = (soma_bbd.select(sum('volume')).collect()[0][0])

print(f'Soma do volume BBD: {soma_volumes_bbd}')

Soma do volume BBD: 779574154.0


# Questão 5 - Faça a exportação do arquivo .csv pela API, crie uma tabela e a importe o .csv das ações de TSLA - Tesla em um banco PostsgreSQL.<a id="5"></a>

O código abaixo, salva o arquivo (TSLA.CSV) das ações TSLA, na pasta do projeto.

In [14]:
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=TSLA&apikey={key}&datatype=csv'

response = requests.get(url)

with open(r'TSLA.csv', 'w') as file:
    file.write(response.text)


 Após isso, é executado dentro do banco de dados Postgre, a criação da tabela "tsla" com a seguinte query SQL:
 
 ```
 CREATE TABLE tsla(
   timestamp Timestamp, 
   open float, 
   high float, 
   low float, 
   close float, 
   adjusted_close float, 
   volume float, 
   dividend_amount float, 
   split_coefficient float)
```

Em seguida, o CSV exportado da API é transformado em um dataframe. Logo após, é criada a configuração de conexão para realizar a carga de dados no banco de dados PostgreSQL. Esse banco de dados foi configurado em um ambiente local, utilizando um container docker. O arquivo **docker-compose.yml** também está disponível na pasta do projeto.

In [58]:
df = spark.read.csv("TSLA.csv", header=True, inferSchema=True)

postgres_url = "jdbc:postgresql://192.168.1.12:5432/datarisk_desafio"
postgres_user = "admin"
postgres_password = "admin"
postgres_table = "tsla"

df.write \
    .format("jdbc") \
    .option("url", postgres_url) \
    .option("dbtable", postgres_table) \
    .option("user", postgres_user) \
    .option("password", postgres_password) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()


# Considerações

### O desafio proposto não apresenta um alto nível de complexidade, no entanto, pude colocar em prática conhecimentos básicos que são fundamentais para o dia a dia de um engenheiro de dados. Mesmo sendo um desafio simples, foi uma oportunidade valiosa para aprimorar habilidades e ampliar o meu repertório na área.

### Gostaria de expressar minha gratidão a toda equipe DataRisk pela oportunidade de participar do desafio e aplicar meus conhecimentos, muito obriogado.

##### Filipe Carvalho de Jesus - filipe.cdj@gmail.com